In [16]:
import numpy as np
import pandas as pd
import glob 
import os 
import matplotlib.pyplot as plt
import random as rand
from pathlib import Path, PureWindowsPath
import subprocess

In [17]:
'''Makes 3-D Unit Vectors'''
def Vec(theta, phi): 
    vec = np.array([np.cos(phi)*np.sin(theta), np.sin(theta)*np.sin(phi), np.cos(theta)])
    return vec

In [18]:
'''Check whether vectors are parallel or orthogonal'''
def isParallel(v1,v2, tolerance=1e-6):
    try:
        if abs(np.dot(v1,v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))  > 1 - tolerance:
            return True
    except ZeroDivisionError:
        return True
    return False

def isOrthogonal(v1,v2, tolerance=1e-6):
    try:
        if abs(np.dot(v1,v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)) < tolerance:
            return True
    except ZeroDivisionError:
        return True
    return False

In [20]:
'''Write a MUMAX3 File and prepare the file directory as necessary'''
def FMR_Thin_Film_Config_Filemaker(B=2000, thetaB=np.pi/2, phiB=0, K=100000, thetaK=np.pi/2, phiK=0,
            aspectRatio=1, PBC=[0,0,0], gridsize=[512,512,4], cellsize=[4,4,4], recorded=False,
            Msat=8e5, Aex=1.3e-11, alpha=0.008, tStep=5e-13, runtime=1e-9, solver=5, smoothing=8, 
            cache=r"C:\\Users\\Rob\\Desktop\\Research\\FMR_Paper_2.0\\Starting_Mag_Cache\\Starter_Magnetization.ovf"):
    '''
    B[int], thetaB/phiB[floats]     External Field (mT) vector in sperical coordinates (radians)
    K[int], thetaK/phiK[floats]     Anisotropy Field (J/m^3) vector in sperical coordianats
    aspectRatio[int (rec:2^n)]      Scales the y-direction of the thin film by factor of 1/input(MUCH faster if 2^n)
    PBC = [list of 3 ints],         Periodic Boundary Conditions, number of periodic repetitions in each direction
    gridsize = [list of  3 ints]    Number of cells in each direction
    cellsize = [list of 3 ints]     Side lengths (in meters) of a cuboidal cell
    Msat[float]                     Saturation Magnetization in J/m^3
    Aex[float]                      Exchange constant in J/m
    alpha[float]                    Damping constant (unitless)
    solver [int]                    Type of solver used by Mumax (1:Euler, 2:Adaptive Heun, 3:RK23 (Bogacki-Shampine),
                                        4:Classical 4th-order Runge-Kutta, 5:RK45 (Dormand-Prince))
    tStep[float]                    Timestep (seconds), important for Fourier Transform
    runtime[float]                  Length of simulation (seconds), choose to be just the region ofexponetial decay
    smoothing[int]                  Edge smoothing factor, integers 0-8
    cache[string(directory)]        Folder containing starting magnetization, as a string
                                        Use the relaxed magnetization from previous simulation to speed up relax()
                                        Relaxing to equilibrium is a CRITICAL step, and is typically the longest 
                                        single step in using the FMR Ringdown method. By using the cache speed ups 
                                        of 10x or more are possible for certain fields and anisotropies.
    '''
    
    '''Make B and K unit vectors'''
    vB = Vec(thetaB,phiB)
    vK = Vec(thetaK,phiK)

    '''Pre-Ringown B Field'''
    vB0 = Vec(thetaB + rand.randrange(1,10)*1e-2, phiB + rand.randrange(1,10)*1e-2)
    
    '''File Naming Scheme:
       External Field'''
    if thetaB == 0:
        fieldPath = 'Out_of_Plane_Field'
    elif thetaB == np.pi/2:
        fieldPath = 'In_Plane_Field'
    else: 
        fieldPath = 'Off_Axis_Field'
   
    '''Anisotropy Field'''
    if thetaK == 0:
        anisPath = 'Out_of_Plane_Anis'
    elif thetaK == np.pi/2:
        anisPath = 'In_Plane_Anis'
    else: 
        anisPath = 'Angular_Anis'
        
    '''Easy or Hard axis'''
    if isParallel(vB,vK):
        axisPath = 'Easy_Axis'
    elif isOrthogonal(vB,vK):
        axisPath = 'Hard_Axis'
    else:
        axisPath = 'Neither'
        
    '''Prepare Directory'''
    basePath = os.getcwd()
    folderpath = os.path.join(basePath, fieldPath, anisPath, axisPath)
#     print(folderpath)
    try:
        os.makedirs(folderpath)
    except OSError:
        pass
    
    '''3-D magnetization recording'''
    if recorded:
        recorder = 'autosave(m, 1e-12);'
    else:
        recorder = ''
        
    '''Prepare config file'''
    lines = [
        'SetGridSize({0}, {1}, {2});'.format(gridsize[0], gridsize[1]/aspectRatio, gridsize[2]),
        'SetPBC({0}, {1}, {2});'.format(PBC[0],PBC[1],PBC[2]),        
        'SetCellSize({0}e-9, {1}e-9, {2}e-9);'.format(cellsize[0], cellsize[1], cellsize[2]),
        'Msat  = {};'.format(Msat),
        'Aex   = {};'.format(Aex),
        'alpha = {};'.format(alpha),
        'edgesmooth = {};'.format(smooothing),
        'B := {}e-3;'.format(B),
        'anisU = vector({0}, {1}, {2});'.format(vK[0], vK[1], vK[2]),
        'Ku1 = {};'.format(K),
        'setSolver({});'.format(solver),
        'TableAdd(Ku1)',
        'TableAdd(B_ext);',
        'TableAutoSave({});'.format(tStep),
        '{}'.format(recorder),
        '//Load starting magnetization',
        'm.loadfile({})'.format(cache),
        'B_ext = vector(B*{0}, B*{1}, B*{2});'.format(vB0[0], vB0[1], vB0[2]),
        'relax();',
        'save(m);',
        'MaxDt = {};'.format(tStep),
        'B_ext = vector(B*{0}, B*{1}, B*{2});'.format(vB[0], vB[1], vB[2]),
        'run({});'.format(runtime),
    ]

    '''Prepare Filename''' 
    filename = 'ThinFilmFMR_'
    filename += 'B({0},{1},{2})_'.format(B, thetaB, phiB)
    filename += 'K({0},{1},{2})_'.format(K, thetaK, phiK)
    filename += 'x-yRatio(1-{})'.format(aspectRatio)
    filename += '.mx3'
  
    '''Write Config File'''
    full_filename = os.path.join(basePath, fieldPath, anisPath, axisPath, filename)
    with open(os.path.join(basePath, fieldPath, anisPath, axisPath, filename), 'w') as f:
        f.writelines("%s\n" % l for l in lines)
    


In [24]:
'''Test the writer for all types of outputs'''

def Thin_Film_Config_Test():
    '''In-plane field, with parallel in-plane anisotropy, EASY'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/2, phiB=0, thetaK=np.pi/2, phiK=0)
    '''In-plane field, with perpendicular in-plane anisotropy, HARD'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/2, phiB=0, thetaK=np.pi/2, phiK=np.pi/2)
    '''In-plane field, with perpendicular Out-of-plane anisotropy, HARD'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/2, phiB=0, thetaK=0, phiK=np.pi/2)
    '''In-plane field with, in-plane anisotropy, NEITHER'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/2, phiB=0, thetaK=np.pi/2, phiK=np.pi/4)

    '''Out-of-plane field, with parallel out-of-plane anisotropy, EASY'''
    FMR_Thin_Film_Config_Filemaker(thetaB=0, phiB=0, thetaK=0, phiK=0)
    '''Out-of-plane field, with perpendicular (in-plane) anisotropy, HARD'''
    FMR_Thin_Film_Config_Filemaker(thetaB=0, phiB=0, thetaK=np.pi/2, phiK=0)
    '''Out-of-plane field with angular anisotropy, NEITHER'''
    FMR_Thin_Film_Config_Filemaker(thetaB=0, phiB=0, thetaK=np.pi/4, phiK=0)

    '''Off-axis field, with parallel anisotropy, EASY'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/4, phiB=np.pi/4, thetaK=np.pi/4, phiK=np.pi/4)
    '''Off-axis field, with perpendicular anisotropy, HARD'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/4, phiB=np.pi/4, thetaK=np.pi/2, phiK=3*np.pi/4)
    '''Off-axis field with angular anisotropy, NEITHER'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/4, phiB=np.pi/4, thetaK=np.pi/2, phiK=np.pi/4)
    '''Off-axis field with out-of-plane anisotropy, NEITHER'''
    FMR_Thin_Film_Config_Filemaker(thetaB=np.pi/4, phiB=np.pi/4, thetaK=0, phiK=np.pi/4)

Thin_Film_Config_Test()
    

In [27]:
'''Run the '''
def FMR_Thin_Film_InPlane_InPlane_Easy(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                       theta=np.pi/2, phi=0):
    '''Square thin films, with B and K pointed in the x-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phi, phiK=phi)
            
            
def FMR_Thin_Film_InPlane_InPlane_Hard(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                       theta=np.pi/2, phiB=0, phiK=np.pi/2):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)


def FMR_Thin_Film_OutPlane_OutPlane_Easy(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                         theta=0, phi=0):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)
                        

def FMR_Thin_Film_OutPlane_OutPlane_Easy(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                         theta=0, phi=0):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)

            
def FMR_Thin_Film_OutPlane_OutPlane_Easy(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                         theta=0, phi=0):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)

            
def FMR_Thin_Film_OutPlane_OutPlane_Easy(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                         theta=0, phi=0):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)
            
            
def FMR_Thin_Film_OutPlane_InPlane_Hard(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                         theta=0, theta=np.pi/2, phi=0):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)
            
            
def FMR_Thin_Film_OutPlane_InPlane_Hard(b_min=0, b_max=2000, b_step=10, k_min=0, k_max=100000, k_step=25000,
                                         theta=0, theta=np.pi/2, phi=0):
    '''Square thin films, with B in the x-direction, K in the y-direction'''
    b_strength = np.arange(b_min, b_max+1, b_step)
    k_strength = np.arange(k_min, k_max+1, k_step)
    for B in b_strength:
        for K in k_strength:
            FMR_Thin_Film_Config_Filemaker(B=B, K=K, thetaB=theta, thetaK=theta, phiB=phiB, phiK=phiK)

In [ ]:
class stuff:
    def __init__(self, val):
        self.val = val
        
    def fmr_1(val):
        
        
stuff().fmr_1
   

Make In Plane Field, out of plane anis

Out of Plane field, in plane anis

Out of plane field, out of plane anis
